# Hahow 課程資料清理程式

本程式共分為兩個部分：

### **PART 1 | 分類資料清理**
本部分會逐一讀取各分類的 CSV 檔案（來自 `merged_data\` 資料夾），針對尚未結構化的「課程名稱」欄位進行拆解與清理。
透過正則表示法（Regular Expression），從課程名稱中提取以下 7 個欄位資訊：
 - 課程名（不含後綴）
 - 講師
 - 評分
 - 評價數
 - 課程時長（統一轉換為「小時」）
 - 購買人數
 - 價錢

此外，也會將數值欄位（如：時長、購買人數、價格）統一轉換為可分析的數值型態，並將課程連結補全為完整網址。

✅ 清理後的每一分類資料，會儲存於資料夾：`cleaned_data\` 中，檔名格式為 `📄 hahow_{分類名稱}_cleaned.csv`。

##### **📌 原始資料範例（未清理前）：**

課程名稱：限時優惠課程唱出你的特色！蔡佩軒的歌唱訓練課by蔡佩軒 Ariel4.9(32)7.5 小時2,382NT$6,300
 
##### **🔍 清理後拆解結果：**

 - 課程名：唱出你的特色！蔡佩軒的歌唱訓練課  
 - 講師：蔡佩軒 Ariel  
 - 評分：4.9  
 - 評價數：32  
 - 課程時長：7.5   
 - 購買人數：2382  
 - 價錢：6300

### **PART 2 | 合併清理後資料**
當所有分類都完成清理後，程式會自動將 `cleaned_data\` 中的檔案合併為一份總表：`📄 hahow_all_courses.csv`

✅ 該檔案會儲存於資料夾：`total_data\`，方便後續進行統一分析與視覺化處理（如：Power BI、Tableau 或 Python 繪圖）。


## **Part 1 分類資料清理**

#### 模組匯入區

In [7]:
import os
import pandas as pd 

#### 自定函數區 — 轉換型態 

In [8]:
# 將各欄位資料轉換為正確的型態，去除空白並統一文字格式

# 處理「評價數」欄位 - 去除「k」字並轉換成數字且乘以 1000（如人數數量 : 2k → 2000）
def convert_review(value):
    if pd.isna(value):
        return 0
    if isinstance(value, str) and "k" in value:
        return float(value.replace("k", "")) * 1000
    else:
        return float(value)

# 處理「課程時長」欄位 - 去除「小時」或「分鐘」字，且換成「小時」單位 （如: 30分鐘 → 0.5 / 1小時 → 1）
def convert_time(value):
    if isinstance(value,str):
        if "分鐘" in value:
            return float(value.replace("分鐘",""))/60
        else:
            return float(value.replace("小時",""))
    return None

# 處理「價錢」欄位 - 去除空白且換成數值型態
def convert_price(value):
    if pd.isna(value):
        return 0
    else:
        return float(value.replace(",", ""))

# 處理「購買人數」欄位 - 去除空白且換成數值型態
def convert_buyers(value):
    if pd.isna(value):
        return 0
    else:
        return float(value.replace(",", ""))

#### 自定函數區 — 將課程名稱拆分多欄位

In [9]:
# 利用正則將課程名拆分多欄位，再轉換各欄位的數值型態和文字格式

def hahow_clean(data):
    
    # 拆分課程名稱
    pattern1 = r"課程(.+)by"
    data["課程"] = data["課程名稱"].str.extract(pattern1)

    # 拆分講師
    pattern2 = r"by(.+?)(?=\d)"
    data["講師"] = data["課程名稱"].str.extract(pattern2)

    # 拆分評分
    pattern3 = r"(\d+\.\d+)(?=\()" 
    data["評分"] = data["課程名稱"].str.extract(pattern3)
    data["評分"] = data["評分"].fillna(0).astype(float)

    # 拆分評價數
    pattern4 = r"\((\d+|\d+\.\d+k)\)" 
    data["評價數"] = data["課程名稱"].str.extract(pattern4).iloc[:, 0].str.strip()
    data["評價數"] = data["評價數"].apply(convert_review)

    # 拆分課程時長
    pattern5 = r"((\d+\.\d+|\d+)\s*(小時|分鐘))"
    extracted = data["課程名稱"].str.extract(pattern5)
    data["課程時長"] = extracted[1] + extracted[2].str.strip()
    data["課程時長"] = data["課程時長"].apply(convert_time).round(2)

    # 拆分購買人數
    pattern6 = r"([\d,]+)(?=\s*NT|免費)"
    data["購買人數"] = data["課程名稱"].str.extract(pattern6).iloc[:, 0].str.strip()
    data["購買人數"] = data["購買人數"].apply(convert_buyers)

    # 拆分價錢
    pattern7 = r"NT\$([\d,]+)"
    data["價錢"] = data["課程名稱"].str.extract(pattern7).iloc[:, 0].str.strip()
    data["價錢"] = data["價錢"].apply(convert_price)

    # 補全連結
    data["link"] = "https://hahow.in" + data["link"]

#### 程式設定區

In [10]:
# 定義課程類別與對應網址代碼

course_type = {
    "音樂": "music",
    "語言": "language",
    "攝影": "photography",
    "藝術": "art",
    "設計": "design",
    "人文": "humanities",
    "行銷": "marketing",
    "程式": "programming",
    "投資理財": "finance-and-investment",
    "職場技能": "career-skills",
    "手作": "handmade",
    "生活品味": "lifestyle"
}

#### 清理主程式區

In [11]:
# 建立資料夾
folder_name = "cleaned_data"

# 如果資料夾不存在就建立
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    print("已建立cleaned_data資料夾")

# 逐一清理每個分類
for category_name in course_type:
    folder = "merged_data"
    file_name = f"{folder}/hahow_{category_name}_merged.csv" 
    data = pd.read_csv(file_name, encoding="utf-8-sig")

    # 進行資料清理
    hahow_clean(data)

    # 重新安排欄位順序
    columns = ["分類", "熱門標籤", "課程", "講師", "評分", "評價數", "課程時長", "購買人數", "價錢", "link", "課程名稱"]
    data = data.reindex(columns=columns)

    # 刪掉課程名稱欄位
    data.drop(columns = ["課程名稱"],inplace=True)
    
    # 刪掉 NAN 資料列
    data.dropna(inplace=True)
    print(f"{category_name} 清理完成")
    
    # 輸出清理完的 CSV
    filename = os.path.join(folder_name,f"hahow_{category_name}_cleaned.csv") 
    data.to_csv(filename, encoding="utf-8-sig", index=False)
    print(f"hahow_{category_name}_cleaned.csv 輸出完成")

print("全部分類完成!!")

已建立cleaned_data資料夾
手作 清理完成
hahow_手作_cleaned.csv 輸出完成
音樂 清理完成
hahow_音樂_cleaned.csv 輸出完成
攝影 清理完成
hahow_攝影_cleaned.csv 輸出完成
全部分類完成!!


## **Part 2 合併清理後資料**

#### 程式設定區

In [ ]:
# 定義課程類別與對應網址代碼

course_type = {
    "音樂": "music", 
    "語言": "language",
    "攝影": "photography", 
    "藝術": "art",
    "設計": "design", 
    "人文": "humanities",
    "行銷": "marketing",
    "程式": "programming", 
    "投資理財": "finance-and-investment",
    "職場技能": "career-skills", 
    "手作": "handmade",
    "生活品味": "lifestyle"
}

#### 合併主程式區

In [12]:
#將12個分類檔完整合併一個檔案
import os
import pandas as pd

# 建立資料夾
folder_name = "total_data"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    print("已建立total_data資料夾")

# 合併各分類清理資料
all_data = [] #存放合併資料

for category_name in course_type:
    folder = "cleaned_data"
    file_path = f"{folder}/hahow_{category_name}_cleaned.csv" 

    # 確認資料是否存在
    if os.path.exists(file_path):
        data = pd.read_csv(file_path,encoding="utf-8-sig")
        all_data.append(data) #資料存入
        print(f"hahow_{category_name}_cleaned.csv 讀取成功")
    else:
        print(f"找不到{file_path}，略過此分類")



# 匯出合併後的 CSV
if all_data: # 確認有資料才做合併
    all_data_df = pd.concat(all_data,ignore_index=True)
    filename = os.path.join(folder_name,"hahow_all_courses.csv")
    all_data_df.to_csv(filename,encoding="utf-8-sig",index=False)
    print("hahow_all_courses.csv 輸出完成")
else:
    print("沒有可合併的資料")   

已建立total_data資料夾
hahow_手作_cleaned.csv 讀取成功
hahow_音樂_cleaned.csv 讀取成功
hahow_攝影_cleaned.csv 讀取成功
hahow_all_courses.csv 輸出完成
